## Establish a connection with our Neptune gremlin endpoint

In [2]:
from __future__  import print_function  # Python 2/3 compatibility

from gremlin_python import statics
from gremlin_python.structure.graph import Graph
from gremlin_python.process.graph_traversal import __
from gremlin_python.process.traversal import T
from gremlin_python.process.strategies import *
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection

statics.load_statics(globals())

graph = Graph()
g = graph.traversal().withRemote(DriverRemoteConnection('ws://neptunedbcluster-umztptccpt2t.cluster-ct6mcfcdqyrw.us-east-1.neptune.amazonaws.com:8182/gremlin','g'))

## Define some helper functions for gremlin operations

In [3]:
def findCustomer(name):
    return g.V().hasLabel('customer').has('name', name).valueMap(True).toList()[0]

In [4]:
def findProduct(title):
    return g.V().hasLabel('product').has('product_title', title).valueMap(True).toList()[0]

In [5]:
def addReview(customer, product, rating):
    fromId = customer[T.id]
    toId = product[T.id]
    return g.V(fromId).addE('review').to(g.V(toId)).property('star_rating', rating).toList()

In [6]:
def recommend_products(customer_id):
    return g.V(customer_id).as_('Target').\
    sideEffect(out('review').store('ProductsTargetAlreadyReviewed')).\
    outE('review').has('star_rating', 5).inV().\
    inE('review').has('star_rating', 5).outV().\
    where(neq('Target')).\
    outE('review').has('star_rating', 5).inV().\
    where(without('ProductsTargetAlreadyReviewed')).\
    dedup().\
    values('product_title').\
    toList()


## Find and query some pre-existing demo customers added out-of-band

In [7]:
gabe = findCustomer('gabe')
cam = findCustomer('cam')

In [8]:
recommend_products(gabe[T.id])

['CS Go']

## Count products, customers, and reviews

In [9]:
# How many products?
g.V().hasLabel('product').count().toList()

[65845]

In [10]:
# How many customers?
g.V().hasLabel('customer').count().toList()

[1045775]

In [11]:
# How many reviews?
g.E().hasLabel('review').count().toList()

[1786052]

## Fetch product recommendations from the full imported reviews dataset for a customer with lots of reviews 

In [12]:
recommend_products('c38752775') # This user reviewed a lot of games

['Mortal Kombat Advance',
 'The Suffering',
 'Super Mario World: Super Mario Advance 2',
 'F-Zero: Maximum Velocity',
 'WCW/NWO Revenge',
 'Dragon Warrior III',
 'Dragon Warrior III',
 'Project Gotham Racing',
 'Playstation 2 Console - Black',
 "Tony Hawk's Pro Skater 2",
 'Saturday Night Slammasters',
 'Half-Life 2: Episode One',
 'Teenage Mutant Ninja Turtles III: The Manhattan Project',
 'God of War - PlayStation 2',
 'Advance Wars',
 'Double Dragon II: The Revenge',
 'Super Mario Advance',
 'Super NES System',
 'Super Mario Bros. 3',
 'Star Wars - Episode I - Racer',
 'XCOM: Enemy Unknown',
 "Spider-Man: Mysterio's Menace",
 'WCW vs. NWO: WorldTour',
 'Half-Life - PC',
 'Teenage Mutant Ninja Turtles III, The Manhattan Project',
 'Sonic Mega Collection',
 'Elder Scrolls III: Morrowind',
 'Grand Theft Auto: Vice City (Jewel Case)',
 'Half-Life 2: Episode Pack - PC',
 "Tony Hawk's Pro Skater 2 - PC",
 'RF Switch/RF Modulator',
 'Mortal Kombat Trilogy',
 'Mortal Kombat Trilogy',
 'Supe